In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))
import psycopg2
import boto3
import csv
import SomewhatSecretStuff as sss

In [ ]:
#this stuff changes for file to uplaod
localFolder = './data' #'E:/Box Sync/Vesicle Deformability/Results/Final Versions/2017_09_10 - SOPC old 1um GFP - 4 (t4-b2-a1)'
localFile = 'cellData_2017-09-12_02-16-31'
inDelim = ','
inExt = 'csv'

hasHeader = False
tableName = 'prod.cell_data'

#operates in place to transform one raw data row to into format suitable for COPY FROM
experimentNo = 4
def process_row(row):
    row[8] = str(10.0 if row[8] == '1.#J' else min(10.0, float(row[8])))  #fix aspect ratio really large or infinite
    row[9] = str(min(1.0, float(row[9])))  #circularity should be <= 1
    if len(row) > 14:
        del row[14] # remove blank column at end?
    del row[6]  #remove Spd column
    row.insert(0,experimentNo)  #prepend extra column for experiment #

processRowFunc = process_row

In [63]:
#this stuff changes if loading to different AWS stuff
s3BucketName = 'mgrigola-redshift-upload'
keyId = sss.AWSkeyId  #this would be poor design for real world system (is secret)
secretKey = sss.AWSsecretKey #i think you're supposed to use environment vars or IAM something something. not this

#Host is given at Redshift > Clusters > [choose cluster] > [Endpoint] (minus the port#)
#database is created in query editor? redhsift gives you a db 'dev' and you connect to that and CREATE DATABASE somthingelse
connData = {
    'host':'microfluidics-data.cin96tvuwjcv.us-east-1.redshift.amazonaws.com',
    'port':5439,
    'dbname':'research_data',
    'user':'mgrigola',
    'password':sss.mgrigolaUserPass
}

#this stuff could change... mostly irrelevant config stuff
localTempFolder = 'C:/temp'
tempDelim = '\t'
tempExt = 'tsv'


#calculate some extended/full file names from the parts that cahnge above
localFullPath = localFolder+'/'+localFile+'.'+inExt
localFullTempPath = localTempFolder+'/'+localFile+'.'+tempExt
awsFileName = 'temp/'+localFile+'.'+tempExt # how S3 sees its file names (given the bucket, no base portion needed)
awsFullFile = 's3://'+s3BucketName+'/'+awsFileName # s3://bucket/folder/file = how redshfit sees awsFileName in S3

print('transforming file...')
success = False

#read original file, transform data to match aws table format, copy to local file (original unchanged, we upload the copy)
rawColNames = {}
with open(localFullPath,'r') as csvFile:
    csvReader = csv.reader(csvFile, delimiter=inDelim)
    header = next(csvReader)  #get/skip header
    rawColNames = {colName:colNo for colNo, colName in enumerate(header) if colName != ''} #lookup dict -> header:columnNumber
    with open(localFullTempPath,'w') as writeFile:
        csvWriter = csv.writer(writeFile, delimiter=tempDelim, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in csvReader:
            if processRowFunc: processRowFunc(row)
            csvWriter.writerow(row)
        
        success = True


if not success:
    print('failed to read/transform file - check local file name')
else:
    print('uploading to S3...')
    success = False

    #upload our local file to s3
    #apparently can also use 'AWS CLI' for credentials, or IAM? or set some environment variables: AWS_ACCESS_KEY_ID / AWS_SECRET_ACCESS_KEY
    # with boto3.Session(aws_access_key_id=keyId, aws_secret_access_key=secretKey) as session:
    #     s3 = session.resource('s3')
    #     s3.meta.client.upload_file(localFullTempPath, s3BucketName, awsFileName)
    session = boto3.Session(aws_access_key_id=keyId, aws_secret_access_key=secretKey)
    s3 = session.resource('s3')
    s3.meta.client.upload_file(localFullTempPath, s3BucketName, awsFileName)
    success = True


if not success:
    print('failed to upload to S3? Check if file exists in S3. Maybe access key is wrong?')
else:
    print('copying to Redshift...')
    success = False
    
    headerRows = 0 if not hasHeader else int(hasHeader)
    
    #open connection to redshift cluster + database.
    with psycopg2.connect(**connData) as conn:
        #conn = psycopg2.connect(**connData)  #this snazzy syntax, same as passing each item in the dict like key='value'
        cur = conn.cursor()

        #the SQL query that runs on Redshift. Care with the quotes. This COPY FROM will append the new data (not overwrite). Duplicate/existing key will yield key error and failure
        copyStatement = "\
        COPY {} FROM '{}' DELIMITER '{}' \
        CREDENTIALS 'aws_access_key_id={};aws_secret_access_key={}' \
        IGNOREHEADER {}  TRUNCATECOLUMNS  ROUNDEC  NULL AS '1.#J' \
        ;".format(tableName, awsFullFile, tempDelim, keyId, secretKey, headerRows)  #can also commit here:  <query>; commit;

        cur.execute(copyStatement)
        #print('ok?', conn.status) #psycopg2.extensions.STATUS_READY=1 #psycopg2.extensions.STATUS_IN_TRANSACTION=2 # i don't get the status...
        conn.commit()  #need the commit here! not autocommitted like thru editor
        #conn.close()  #don't need this if doing the with thing
        success = True

if not success:
    print('something went wrong...')
else:
    print('all good!')

transforming file...
uploading to S3...
copying to Redshift...
all good!


In [66]:
import boto3

localFileName = './data' #'C:/Users/m/Desktop/AWS/test_copy.csv'

targetFileName = 'test_copy.csv'

session = boto3.Session(aws_access_key_id=keyId, aws_secret_access_key=secretKey)
#client = boto3.client('s3', aws_access_key_id=keyId, aws_secret_access_key=secretKey)
s3 = session.resource('s3')

s3.meta.client.upload_file(localFileName, bucketName, targetFileName)

#bucket = s3.Bucket('mgrigola-redshift-upload')
#myFile = s3.Object(bucket_name='mgrigola-redshift-upload', key='test/cellData_2017-09-12_01-04-24.csv')

In [ ]:
inFile = 'E:/Box Sync/Vesicle Deformability/Results/Final Versions/2017_09_10 - SOPC old 1um GFP - 2 (t4-b2-a1)/cellData_2017-09-12_01-50-00.csv'
outFile = 'C:/temp/cell_data.tsv'
inDelim = ','
outDelim = '\t'
experimentNo = 1

#operates in place
def process_row(row, expNo):
    row[8] = str(10.0 if row[8] == '1.#J' else min(10.0, float(row[8])))  #fix aspect ratio really large or infinite
    row[9] = str(min(1.0, float(row[9])))  #circularity should be <= 1
    if len(row) > 14:
        del row[14] # remove blank column at end?
    del row[6]  #remove Spd column
    row.insert(0,expNo)  #prepend extra column for experiment #


colNames = {}
with open(inFile,'r') as csvFile:
    csvReader = csv.reader(csvFile, delimiter=inDelim)
    header = next(csvReader)  #get/skip header
    colNames = {colName:colNo for colNo, colName in enumerate(header) if colName != ''} #lookup dict -> header:columnNumber
    with open(outFile,'w') as writeFile:
        csvWriter = csv.writer(writeFile, delimiter=outDelim, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in csvReader:
            process_row(row, experimentNo)
            csvWriter.writerow(row)

In [ ]:
import pandas as pd

fileName = 'E:/Box Sync/Vesicle Deformability/Results/Final Versions/2017_09_10 - SOPC old 1um GFP - 1 (t4-b2-a1)/cellData_2017-09-12_01-04-24.csv'

df = pd.read_csv(fileName, sep=',', header=[0], na_values=['1.#J'])
if 'Unnamed: 14' in df.columns:
    df = df.drop('Unnamed: 14',1)
if 'Spd' in df.columns:
    df = df.drop('Unnamed: 14',1)
df['Aspc'] = df['Aspc'].apply(lambda x: min(x,10.0))
df['Circ'] = df['Circ'].apply(lambda x: min(x,1.0), )

In [ ]:
#alternative connection syntax
cHost = 'microfluidics-data.cin96tvuwjcv.us-east-1.redshift.amazonaws.com'
cUser = 'mgrigola'
cPort = 5439
cPass = sss.mgrigolaUserPass
cDbName = 'research_data'
conn = psycopg2.connect(host=cHost, user=cUser, port=cPort, password=cPass, dbname=cDbName)

#globalFileName = 'https://s3.amazonaws.com/mgrigola-redshift-upload/2017_09_10+-+SOPC+old+1um+GFP+-+1+(t4-b2-a1)/cellData_2017-09-12_01-04-24.csv'
#fileName = 's3://mgrigola-redshift-upload/2017_09_10 - SOPC old 1um GFP - 1 (t4-b2-a1)/cellData_2017-09-12_01-04-24.csv'
#awsFileName = 's3://mgrigola-redshift-upload/test/cellData_2017-09-12_01-04-24.csv'